<a href="https://colab.research.google.com/github/dimsparagis0210/MLTask/blob/main/ML_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Απαλλακτική εργασία στο μάθημα Μηχανική Μάθηση**

**Σπαράγγης Δημήτρης (ics22066)**
</br>
**Βιτσιώτη Μαγδαληνή (ics22030)**